# COMP47750 Machine Learning Assignment
# Gaussian Naive Bayes
A reimplementation of the `sklearn` Gaussian Naive Bayes classifier. 

1. Provide a python class MyGaussianNB that implements Gaussian Naive Bayes. 
The API specification for sklearn classifiers is here: https://scikit-learn.org/stable/developers/develop.html 
You should implement the ‘fit’ and ‘predict’ methods, there is no need to implement ‘predict_proba’. 


In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from collections import Counter
from sklearn.metrics import accuracy_score

## My GaussianNB
Reimplementation of a Gaussian Naive Bayes.

In [ ]:
class MyGaussianNB(BaseEstimator, ClassifierMixin):          
    def fit(self, Xt, yt):
        self.var_smoothing = 1e-9
        self.Xt = Xt
        self.yt = yt
        self.n_feat = Xt.shape[1]
        # print(Xt)  how many rows and columns there are (4 columns)
        # print(Xt.shape[1])  output 4
        self.mus = {}
        self.sig_sqs = {}
        self.priors = {}
        self.number = 0
        
        c_dict = Counter(self.yt)
        # print(c_dict)  Counter({'Adelie': 76, 'Gentoo': 56, 'Chinstrap': 34})
        # Mus and sig_sqs are stored by calculating their four characteristics for different result categories.
        
        for c in c_dict.keys():
            self.mus[c] = np.zeros(self.n_feat) # where the means will be stored
            self.sig_sqs[c] = np.zeros(self.n_feat) # where the variances will be stored
            self.priors[c] = c_dict[c]/Xt.shape[0]
            
            mask = self.yt == c
            X_tr_c = self.Xt[mask, :] # the rows for this class label
            # print(X_tr_c)
            
            for f in range(self.n_feat):
                
                # Determine whether it is nan or not,nan value will be ignored
                lst = filter(lambda x: np.isnan(x) == False, X_tr_c[:,f])
                lst = list(lst)
                self.number = self.number + (len(X_tr_c[:,f]) - len(lst))
                # self.number is a quantity of missing values
                print(self.number)
                                                
                self.mus[c][f] = np.mean(np.array(lst))
                self.sig_sqs[c][f] = np.var(np.array(lst) + self.var_smoothing)
        
        # The limit warning on the number of missing values allowed
        if (self.number >= 25):
            print("After calculation, there are total " + str(self.number) + " missing values")
            print("Warning: there are to many missing values, Please try again.")
        else:
            print("There are missing values less than 25.")
        
        return self
    
    # this function is to verify whether it's a number
    def is_number(self, s):
        try:
            float(s)
            return True
        except ValueError:  
            pass  
        try:
            import unicodedata 
            unicodedata.numeric(s)  
            return True
        except (TypeError, ValueError):
            pass
        return False
    
    # The predictions are the most common class in the training set.
    def predict(self, Xtes):
        self.Xtes = Xtes
         
        res_list = []
        for sample in Xtes:
            res_list.append(self.predict_single(sample))
            
        return np.array(res_list)
    
    def predict_single(self, x_single):
        probs = {}
        for c in self.priors.keys():   # for each of the class labels
            probs[c] = self.priors[c]
            for i, f in enumerate(x_single):
                
                # missing values will be set with nan in advance, if it is not nan, 
                # then it will be calculated for posterior probability calculation.
                if(np.isnan(f)== False):
                    t1 = 1/math.sqrt(2*math.pi*self.sig_sqs[c][i])
                    num = (f - self.mus[c][i])**2
                    den = 2*self.sig_sqs[c][i]
                    pxi_y = t1 * math.exp(-num/den)
                    probs[c] = probs[c] * pxi_y
                    #print(t1, num, den, pxi_y)
                    #print(probs)
                    #print(c, self.priors[c])
                else:
                    pass
            
        return max(probs, key=probs.get) # Return the key with the largest value
    

## Testing
2. Test the performance of your implementation against the `GaussianNB` implementation in `scikit-learn`. You should use a range of datasets for this testing.   
Four datasets are used for testing; testing on a hold out set:
 - **penguins**: check that mean and variance estimates are the same, check that predictions are the same. 
 - **diabetes**: check that predictions are the same.
 - **glassV2**: test that predictions are the same. 
 - **bike_sharing**: test that predictions are the same. 

### Penguins

In [ ]:
penguins = pd.read_csv('PenguinsMV0.4.csv', index_col = 0)

# you can uncomment below command to run and test 0.4 data file.
# penguins = pd.read_csv('PenguinsMV0.4.csv', index_col = 0)
print(penguins.shape)
penguins.head()

# Preprocessing the data

In [ ]:
f_names = ['bill_length_mm', 'bill_depth_mm','flipper_length_mm', 'body_mass_g']
penguins = penguins[f_names + ['species']]
penguins = penguins.loc[penguins['species'].isin(['Adelie','Chinstrap'])]
# set species attribute to -1 position and only kepp Adelie and Chinstrap values

In [ ]:
# set all missing values with nan.

def elementchange(list):
    for element in list:
        q = penguins[element].values
        for i in range(len(q)):
            if MyGaussianNB().is_number(q[i]):
                pass
            else:
                penguins[element].values[i] = np.nan


In [ ]:
y = penguins.pop('species').values

In [ ]:
elementchange(f_names)
X_raw = penguins.values

X_tr_raw, X_ts_raw, y_train, y_test = train_test_split(X_raw, y, random_state=2, test_size=1/2)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_tr_raw)
X_test = scaler.transform(X_ts_raw)
max_k = X_train.shape[1]

# X_train = X_tr_raw
# X_test = X_ts_raw

X_train.shape, X_test.shape

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.impute import *
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#GaussianNB using missing value imputation

imp = KNNImputer(missing_values = np.nan)
imp.fit(X_tr_raw)
X_train_G = imp.transform(X_tr_raw)
X_test_G = imp.transform(X_ts_raw)


imp2 = SimpleImputer(missing_values = np.nan, strategy='mean')
imp2.fit(X_tr_raw)
X_train_G2 = imp2.transform(X_tr_raw)
X_test_G2 = imp2.transform(X_ts_raw)


imp3 = IterativeImputer(max_iter=10, random_state=0)
imp3.fit(X_tr_raw)
X_train_G3 = imp3.transform(X_tr_raw)
X_test_G3 = imp3.transform(X_ts_raw)
# scaler = StandardScaler()
# X_train_G = scaler.fit_transform(X_tr_raw)
# X_test_G = scaler.transform(X_ts_raw)
max_k_G = X_train_G.shape[1]

X_train_G.shape, X_test_G.shape

# Pipelines & Cross Validation

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


# the cross validation for KNN imputation
kNNpipe  = Pipeline(steps=[
    ('imputer', KNNImputer(missing_values = np.nan)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])


acc_arr = cross_val_score(kNNpipe, X_raw, y, cv=5, n_jobs = -1)
print("Accuracy: {0:4.2f}".format(sum(acc_arr)/len(acc_arr)))


In [ ]:
# the cross validation for univariate imputation
kNNpipe2  = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values = np.nan, strategy='mean')),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])


acc_arr2 = cross_val_score(kNNpipe2, X_raw, y, cv=5, n_jobs = -1)
print("Accuracy: {0:4.2f}".format(sum(acc_arr2)/len(acc_arr2)))

In [ ]:
# the cross validation for multi-variate imputation
kNNpipe3  = Pipeline(steps=[
    ('imputer', IterativeImputer(max_iter=10, random_state=0)),
    ('scaler', StandardScaler()),
    ('classifier', GaussianNB())])


acc_arr3 = cross_val_score(kNNpipe3, X_raw, y, cv=5, n_jobs = -1)
print("Accuracy: {0:4.2f}".format(sum(acc_arr3)/len(acc_arr3)))

In [ ]:
# the # the cross validation for my approach

mkNNpipe  = Pipeline(steps=[
    ('imputer', elementchange(f_names)),
    ('scaler', StandardScaler()),
    ('classifier', MyGaussianNB())])


macc_arr = cross_val_score(mkNNpipe, X_raw, y, cv=5, n_jobs = -1)
print("Accuracy: {0:4.2f}".format(sum(macc_arr)/len(macc_arr)))

The accuracies for "PenguinsMV0.4.csv" file are 0.80(KNN imputation) and 0.83(my approach) respectively.

The accuracies for "PenguinsMV0.2.csv" file are 0.95(KNN imputation) and 0.95(my approach) respectively.

The accuracies for "PenguinsMV0.2.csv" file are 0.95(univariate imputation) and 0.95(univariate imputation) respectively.

The accuracies for "PenguinsMV0.4.csv" file are 0.83(univariate imputation) and 0.80(univariate imputation) respectively.


**Model Parameters**  
The means are showed below:

In [ ]:
gnb = GaussianNB()
gnb.fit(X_train_G,y_train)

gnb2 = GaussianNB()
gnb2.fit(X_train_G2,y_train)

gnb3 = GaussianNB()
gnb3.fit(X_train_G3,y_train)


mgnb = MyGaussianNB()
mgnb.fit(X_train,y_train)

In [ ]:
mgnb.sig_sqs

In [ ]:
gnb.theta_

In [ ]:
mgnb.mus

Accuracy scores are showed below:

In [ ]:
gnb.score(X_test_G, y_test)

In [ ]:
mgnb.score(X_test, y_test)

The accuracy scores for gnb and mgnb are 0.8037383177570093 and 0.8504672897196262   -------- PenguinsMV0.4.csv

The accuracy scores for gnb and mgnb are 0.9626168224299065 and 0.9626168224299065   -------- PenguinsMV0.2.csv

In [ ]:
gnb2.score(X_test_G2, y_test)


In [ ]:
gnb3.score(X_test_G3, y_test)

In [ ]:
The accuracy scores for gnb2 and gnb3 are 0.8691588785046729 and 0.8504672897196262 -------- PenguinsMV0.4.csv

The accuracy scores for gnb2 and gnb3 are 0.9626168224299065 and 0.9626168224299065 -------- PenguinsMV0.2.csv

### Fidelity tests

Look at the lables of the predictions of the first 10 test samples:

In [ ]:
mgnb.predict(X_test[:15])

In [ ]:
y_test[:15]

In [ ]:
gnb.predict(X_test_G[:15])